# Flood Event Data Processing

This notebook processes storm event CSV files to extract and filter flooding events.


In [1]:
import os
import pandas as pd
from pathlib import Path
from typing import List


## Loading initial data

### Loading data


In [2]:
def get_csv_files(data_dir: str) -> List[str]:
    csv_files = []
    data_path = Path(data_dir)
    
    if not data_path.exists():
        raise FileNotFoundError(f"Directory {data_dir} does not exist")
    
    for file in data_path.glob("*.csv"):
        csv_files.append(str(file))
    
    return sorted(csv_files)


In [3]:
def parse_csv_file(file_path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(file_path, low_memory=False)
        return df
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return pd.DataFrame()


### Filtering


In [4]:
def filter_flooding_events(df: pd.DataFrame, state: str = None) -> pd.DataFrame:
    if df.empty or 'EVENT_TYPE' not in df.columns:
        return pd.DataFrame()
    
    flood_keywords = ['flood', 'flash flood', 'coastal flood', 'lakeshore flood', 
                      'river flood', 'urban flood', 'small stream flood']
    
    # Invert the mask with ~ to select rows that DON'T contain flooding keywords
    mask = ~df['EVENT_TYPE'].str.lower().str.contains('|'.join(flood_keywords), 
                                                       case=False, 
                                                       na=False)    
    if state is not None:
        if 'STATE' not in df.columns:
            print(f"Warning: STATE column not found, cannot filter by state")
        else:
            state_mask = df['STATE'].str.upper() == state.upper()
            mask = mask & state_mask
    
    df = df[mask].copy()

    # Remove rows with invalid coordinates
    if 'BEGIN_LAT' in df.columns and 'BEGIN_LON' in df.columns:
        df['BEGIN_LAT'] = pd.to_numeric(df['BEGIN_LAT'], errors='coerce')
        df['BEGIN_LON'] = pd.to_numeric(df['BEGIN_LON'], errors='coerce')
        mask = (df['BEGIN_LAT'].notna()) & (df['BEGIN_LON'].notna()) & \
            (df['BEGIN_LAT'] != 0) & (df['BEGIN_LON'] != 0)
        
        filtered_df = df[mask].copy()
    else:
        filtered_df = df.copy()

    return filtered_df


In [5]:
def split_yearmonth(df: pd.DataFrame) -> pd.DataFrame:

    if df.empty or 'BEGIN_YEARMONTH' not in df.columns:
        return df
    
    df = df.copy()
    df['YEAR'] = df['BEGIN_YEARMONTH'].astype(str).str[:4].astype(int)
    df['MONTH'] = df['BEGIN_YEARMONTH'].astype(str).str[4:6].astype(int)
    
    return df


In [6]:
def select_columns(df: pd.DataFrame) -> pd.DataFrame:

    if df.empty:
        return pd.DataFrame()
    
    columns_to_keep = [
        'YEAR',
        'MONTH',
        'BEGIN_DAY',
        'BEGIN_TIME',
        'BEGIN_LAT',
        'BEGIN_LON',
        'STATE',
        'EVENT_TYPE',
        'FLOOD_CAUSE',
        'EVENT_NARRATIVE'
    ]
    

    available_columns = [col for col in columns_to_keep if col in df.columns]
    missing_columns = [col for col in columns_to_keep if col not in df.columns]
    
    if missing_columns:
        print(f"Warning: The following columns were not found: {missing_columns}")
    
    if not available_columns:
        print("Warning: None of the requested columns were found in the DataFrame")
        return pd.DataFrame()
    
    selected_df = df[available_columns].copy()
    return selected_df


### Main processing


In [7]:
def process_all_csv_files(raw_data_dir: str, state: str = None) -> pd.DataFrame:
    csv_files = get_csv_files(raw_data_dir)
    
    if not csv_files:
        print(f"No CSV files found in {raw_data_dir}")
        return pd.DataFrame()
    
    print(f"Found {len(csv_files)} CSV file(s) to process")
    if state:
        print(f"Filtering for state: {state.upper()}")
    
    all_flooding_events = []
    
    for csv_file in csv_files:
        print(f"Processing {os.path.basename(csv_file)}...")
        df = parse_csv_file(csv_file)
        
        if not df.empty:
            flooding_df = filter_flooding_events(df, state=state)
            if not flooding_df.empty:
                print(f"  Found {len(flooding_df)} flooding event(s)")
                all_flooding_events.append(flooding_df)
            else:
                print(f"  No flooding events found")
        else:
            print(f"  Failed to parse or file is empty")
    
    if all_flooding_events:
        combined_df = pd.concat(all_flooding_events, ignore_index=True)
        print(f"\nTotal flooding events found: {len(combined_df)}")
        
        combined_df = split_yearmonth(combined_df)
        selected_df = select_columns(combined_df)
        return selected_df
    else:
        print("\nNo flooding events found in any files")
        return pd.DataFrame()


In [8]:
raw_data_dir = "raw_data"
state_filter = "TEXAS" 

print(f"Data directory: {raw_data_dir}")
print(f"State filter: {state_filter if state_filter else 'None (all states)'}")


Data directory: raw_data
State filter: TEXAS


In [9]:
non_flooding_events_df = process_all_csv_files(str(raw_data_dir), state=state_filter)


Found 25 CSV file(s) to process
Filtering for state: TEXAS
Processing StormEvents_details-ftp_v1.0_d2001_c20250520.csv...
  Found 2133 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2002_c20250520.csv...
  Found 2592 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2003_c20250520.csv...
  Found 2311 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2004_c20250520.csv...
  Found 2516 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2005_c20250520.csv...
  Found 2355 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2006_c20250520.csv...
  Found 1932 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2007_c20250520.csv...
  Found 2499 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2008_c20250520.csv...
  Found 2492 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2009_c20250520.csv...
  Found 3002 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2010_c20250520.csv...
  Found 1727 flooding event(s)
P

In [10]:
# Display summary information
if not non_flooding_events_df.empty:
    print(f"\nTotal rows: {len(non_flooding_events_df)}")
    print(f"\nColumns: {list(non_flooding_events_df.columns)}")
    print(f"\nFirst few rows:")
    display(non_flooding_events_df.head(10))
    
    print(f"\nDataFrame info:")
    non_flooding_events_df.info()
else:
    print("No flooding events found.")



Total rows: 57960

Columns: ['YEAR', 'MONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'BEGIN_LAT', 'BEGIN_LON', 'STATE', 'EVENT_TYPE', 'FLOOD_CAUSE', 'EVENT_NARRATIVE']

First few rows:


,YEAR,MONTH,BEGIN_DAY,BEGIN_TIME,BEGIN_LAT,BEGIN_LON,STATE,EVENT_TYPE,FLOOD_CAUSE,EVENT_NARRATIVE
0,2001,11,15,1555,30.93333,-97.53333,TEXAS,Thunderstorm Wind,NaN,There was wind damage to two recreational vehi...
1,2001,11,11,1530,32.65000,-98.90000,TEXAS,Hail,NaN,NaN
2,2001,11,23,2050,33.51667,-96.18333,TEXAS,Thunderstorm Wind,NaN,The roof of a barn was blown off by high winds.
3,2001,12,16,2155,30.68333,-94.18333,TEXAS,Thunderstorm Wind,NaN,NaN
4,2001,12,16,2215,30.65000,-93.91667,TEXAS,Thunderstorm Wind,NaN,NaN
5,2001,12,16,2220,30.68333,-93.81667,TEXAS,Thunderstorm Wind,NaN,NaN
6,2001,1,29,518,30.91667,-94.00000,TEXAS,Thunderstorm Wind,NaN,NaN
7,2001,1,29,545,30.35000,-94.06667,TEXAS,Thunderstorm Wind,NaN,NaN
8,2001,1,29,603,31.00000,-93.63333,TEXAS,Thunderstorm Wind,NaN,NaN
9,2001,12,13,1110,30.05000,-94.88333,TEXAS,Hail,NaN,Liberty Sheriff reported 3/4 inch hail near Da...



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57960 entries, 0 to 57959
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             57960 non-null  int64  
 1   MONTH            57960 non-null  int64  
 2   BEGIN_DAY        57960 non-null  int64  
 3   BEGIN_TIME       57960 non-null  int64  
 4   BEGIN_LAT        57960 non-null  float64
 5   BEGIN_LON        57960 non-null  float64
 6   STATE            57960 non-null  object 
 7   EVENT_TYPE       57960 non-null  object 
 8   FLOOD_CAUSE      1 non-null      object 
 9   EVENT_NARRATIVE  40373 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 4.4+ MB


In [ ]:
# get a random subset of 10000 rows from the non_flooding_events_df
non_flooding_events_df = non_flooding_events_df.sample(n=9500)
non_flooding_events_df.head()


,YEAR,MONTH,BEGIN_DAY,BEGIN_TIME,BEGIN_LAT,BEGIN_LON,STATE,EVENT_TYPE,FLOOD_CAUSE,EVENT_NARRATIVE
44591,2020,4,18,1530,30.67000,-95.53000,TEXAS,Hail,NaN,Softball sized hail was reported.
20409,2009,5,1,1512,33.25960,-99.90000,TEXAS,Tornado,NaN,A trained spotter reported a brief tornado tou...
10693,2005,6,4,2020,28.68333,-99.81667,TEXAS,Thunderstorm Wind,NaN,"Severe thunderstorms struck Crystal City, knoc..."
56753,2025,3,25,2053,33.18000,-96.50000,TEXAS,Hail,NaN,Amateur radio operator reported half dollar si...
19761,2009,3,26,1742,33.80900,-97.73000,TEXAS,Hail,NaN,Half dollar size hail fell 2 miles north of No...


## Augment Data

In [17]:
!pip3 install openmeteo-requests
!pip3 install requests-cache retry-requests

### Augmenting flooding data using the [Open Meteo API](https://open-meteo.com/en/docs/climate-api?utm_source=chatgpt.com&daily=temperature_2m_mean,wind_speed_10m_mean,cloud_cover_mean,relative_humidity_2m_mean,dew_point_2m_mean,precipitation_sum,rain_sum,snowfall_sum,pressure_msl_mean,soil_moisture_0_to_10cm_mean&start_date=2020-01-01&end_date=2020-01-01&models=EC_Earth3P_HR#settings)

This is an open-sourced api that allows you to entire a lat, long, and date and it gives you data on the weather conditions of that area at the time. 

In [12]:
import openmeteo_requests
import requests_cache
from retry_requests import retry

def get_weather_data(lat: float, lon: float, date: str) -> dict:
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    url = "https://climate-api.open-meteo.com/v1/climate"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": date,
        "end_date": date,
        "models": "EC_Earth3P_HR",
        "daily": ["temperature_2m_mean", "wind_speed_10m_mean", "cloud_cover_mean", "relative_humidity_2m_mean", "dew_point_2m_mean", "precipitation_sum", "rain_sum", "snowfall_sum", "pressure_msl_mean", "soil_moisture_0_to_10cm_mean"],
        "utm_source": "chatgpt.com",
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]

    daily = response.Daily()
    daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
    daily_wind_speed_10m_mean = daily.Variables(1).ValuesAsNumpy()
    daily_cloud_cover_mean = daily.Variables(2).ValuesAsNumpy()
    daily_relative_humidity_2m_mean = daily.Variables(3).ValuesAsNumpy()
    daily_dew_point_2m_mean = daily.Variables(4).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(5).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(6).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(7).ValuesAsNumpy()
    daily_pressure_msl_mean = daily.Variables(8).ValuesAsNumpy()
    daily_soil_moisture_0_to_10cm_mean = daily.Variables(9).ValuesAsNumpy()

    daily_data = {"date": date}

    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["wind_speed_10m_mean"] = daily_wind_speed_10m_mean
    daily_data["cloud_cover_mean"] = daily_cloud_cover_mean
    daily_data["relative_humidity_2m_mean"] = daily_relative_humidity_2m_mean
    daily_data["dew_point_2m_mean"] = daily_dew_point_2m_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["pressure_msl_mean"] = daily_pressure_msl_mean
    daily_data["soil_moisture_0_to_10cm_mean"] = daily_soil_moisture_0_to_10cm_mean
    daily_data["elevation"] = response.Elevation()

    daily_dataframe = pd.DataFrame(data = daily_data)
    return daily_dataframe

In [ ]:
# For every flooding event, augment the row with the weather data
# Add the weather data to the flooding data
# Save the augmented data to a new csv file

import time
from datetime import datetime

def augment_flooding_data_with_weather(flooding_df: pd.DataFrame) -> pd.DataFrame:
    augmented_rows = []
    total_rows = len(flooding_df)
    
    print(f"Augmenting {total_rows} flooding events with weather data...")
    print("This may take a while due to API rate limits...\n")
    
    for idx, row in flooding_df.iterrows():
        try:
            year = int(row['YEAR'])
            month = int(row['MONTH'])
            day = int(row['BEGIN_DAY'])
            date_str = f"{year}-{month:02d}-{day:02d}"
            
            weather_df = get_weather_data(
                lat=float(row['BEGIN_LAT']),
                lon=float(row['BEGIN_LON']),
                date=date_str
            )
            
            if not weather_df.empty:
                weather_row = weather_df.iloc[0]
                augmented_row = row.to_dict()
                
                for col in weather_df.columns:
                    if col != 'date':
                        augmented_row[col] = weather_row[col]
                augmented_rows.append(augmented_row)
            else:
                augmented_rows.append(row.to_dict())
            
            if (idx + 1) % 50 == 0:
                print(f"Processed {idx + 1}/{total_rows} events...")
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            augmented_rows.append(row.to_dict())
    augmented_df = pd.DataFrame(augmented_rows)
    
    print(f"\nAugmentation complete! Processed {len(augmented_df)} events.")
    return augmented_df

# Augment the flooding events with weather data
augmented_non_flooding_df = augment_flooding_data_with_weather(non_flooding_events_df)

# Display summary
print(f"\nAugmented DataFrame shape: {augmented_non_flooding_df.shape}")
print(f"\nNew columns added: {set(augmented_non_flooding_df.columns) - set(augmented_non_flooding_df.columns)}")
print(f"\nFirst few rows of augmented data:")
display(augmented_non_flooding_df.head())

# Save to CSV file
output_path = "non_flooding_events_augmented.csv"
augmented_non_flooding_df.to_csv(output_path, index=False)
print(f"\nAugmented data saved to: {output_path}")


Augmenting 9340 flooding events with weather data...
This may take a while due to API rate limits...

Processed 50/9340 events...
Processed 100/9340 events...
Processed 150/9340 events...
Processed 200/9340 events...
Processed 250/9340 events...
Processed 300/9340 events...
Processed 350/9340 events...
Processed 400/9340 events...
Processed 450/9340 events...
Processed 500/9340 events...
Processed 550/9340 events...
Processed 600/9340 events...
Processed 650/9340 events...
Processed 700/9340 events...
Processed 750/9340 events...
Processed 800/9340 events...
Processed 850/9340 events...
Processed 900/9340 events...
Processed 950/9340 events...
Processed 1000/9340 events...
Processed 1050/9340 events...
Processed 1100/9340 events...
Processed 1150/9340 events...
Processed 1200/9340 events...
Processed 1250/9340 events...
Processed 1300/9340 events...
Processed 1350/9340 events...
Processed 1400/9340 events...
Processed 1450/9340 events...
Processed 1500/9340 events...
Processed 1550/934

,YEAR,MONTH,BEGIN_DAY,BEGIN_TIME,BEGIN_LAT,BEGIN_LON,STATE,EVENT_TYPE,FLOOD_CAUSE,EVENT_NARRATIVE,...,wind_speed_10m_mean,cloud_cover_mean,relative_humidity_2m_mean,dew_point_2m_mean,precipitation_sum,rain_sum,snowfall_sum,pressure_msl_mean,soil_moisture_0_to_10cm_mean,elevation
0,2001,3,11,140,32.53333,-96.66667,TEXAS,Flash Flood,NaN,Roads were closed near Sanger due to high water.,...,21.042433,0.000000,39.388832,4.528868,0.0,0.0,0.0,1001.218079,0.432614,146.0
1,2001,5,4,15,33.73333,-102.78333,TEXAS,Flash Flood,NaN,I-27 was flooded and subsequently closed due t...,...,22.688942,3.063162,45.882435,12.775488,0.0,0.0,0.0,1007.632996,0.110644,1150.0
2,2001,5,6,125,31.46667,-97.71667,TEXAS,Flash Flood,NaN,Water was over Northwest Highway at Hillcrest ...,...,13.942765,44.521587,68.509903,18.607435,0.0,0.0,0.0,1016.453674,0.302950,265.0
3,2001,5,6,128,33.08333,-97.13333,TEXAS,Flash Flood,NaN,Water was over Highway 75 at Parkview.,...,12.174953,53.005688,65.300453,18.778494,0.0,0.0,0.0,1016.608337,0.192466,190.0
4,2001,5,6,1700,33.36667,-97.68333,TEXAS,Flash Flood,NaN,Flooding resulted in high water rescues and ma...,...,9.831195,54.140934,63.182789,18.967064,0.0,0.0,0.0,1016.293640,0.186347,280.0



Augmented data saved to: flooding_events_augmented.csv
